## Все данные будем извлекать с двух ресурсов: [fanfics.me](fanfics.me) и [ficbook.ru](ficbook.ru)

Какие вообще данные в теории понадобятся?
- название
- описание фанфика
- персонажи
- пейринги
- фандомы
- возрастной рейтинг
- теги отношений (слеш/джен/гет/фем)
- тег размера (мини/миди/макси/драббл)
- размер
- события (можно получить только для фанфиков с [fanfics.me](fanfics.me))
- законченность
- жанры
- предупреждения
- ссылка

## Что вообще происходит?

Сейчас мы будем доставать все вышеописанные данные с сайта. Для этого нужно открыть страницу со списком фанфиков и достать оттуда все названия, описания, персонажей и так далее. Чтобы произвести эти действия, нужно использовать специализированные библиотеки.

In [1]:
import requests # библиотека для доступа к исходному коду страниц
from bs4 import BeautifulSoup # библиотека для «вытаскивания» нужных нам данных из исходного кода страницы

Каждая страница с фанфикам по Гарри Поттеру на [fanfics.me](fanfics.me) выглядит следующим образом: 
>https://fanfics.me/find?fandom1=2&page=1

где последнее число отвечает за номер страницы. Всего таких страниц с фанфиками 1031 на момент написания этого кода. Давайте вытащим исходный код каждой страницы в цикле.

In [35]:
URLs = ['https://fanfics.me/find?fandom1=2&page=' + str(page) for page in range(1, 3)]
page_sources = [requests.get(url) for url in URLs]

Исходный код надо превратить в «суп», с которым будем удобно работать для поиска информации на странице. «Суп» будем искать для каждой страницы, исходный код которой мы получили. Тем самым получим список с «супами».

In [36]:
soups = [BeautifulSoup(page_source.content, 'html.parser') for page_source in page_sources]

Вся информация о фанфике содержится в контейнере 
```html
<table class='FicTbl'>
```
Поэтому для каждого «супа» найдём все разделы с информацией о фанфиках, тем самым мы получим «суп» для каждого фанфика.

In [75]:
fanfic_soups = []
for soup in soups:
    fanfic_soups += soup.find_all('table', class_='FicTbl')

## Названия
Если внимательно посмотреть на код страницы, то становится ясно, что интересующее нас название лежит в контейнере 
```html
<div class='FicTable_Title'>
``` 
так что ищем этот контейнер (он всего один для каждого «супа», поэтому используем `find` вместо `find_all`)

In [113]:
titles = [soup.find('div', class_='FicTable_Title') for soup in fanfic_soups]
titles[0]

<div class="FicTable_Title">
<h4><a href="/fic71494">Другие берега</a> <span class="small green">(гет)</span></h4>
<div class="FicTable_Stat"><span class="Views" style="margin-right:20px;" title="Количество просмотров 9 871">9.9k</span>
<span class="ReadersCount" style="margin-right:20px;" title="Количество читателей 302">302</span><span class="Comments" style="margin-right:20px;" title="Количество комментариев 6">6</span><span class="RecommendsCount" style="margin-right:20px;" title="Количество рекомендаций">2</span><span class="DateUpdate" style="margin-right:20px;" title="Опубликовано 12.02.2015, изменено 04.02.2021">04.02.2021</span><span class="FanartsCount" style="margin-right:20px;" title="Количество иллюстраций">1</span></div>
</div>

Проблема в том, что контейнер содержит не только название, но и ещё массу лишней информации, поэтому находим уникальный для названия контейнер -- это контейнер `<a>`, он применяется только к названию.

In [114]:
titles = [soup.find('a').text for soup in titles]
titles[0]

'Другие берега'

## Описания
Описание лежит в контейнере 
```html
<td class='FicTbl_sammary'>
``` 
так что ищем этот контейнер (он всего один для каждого «супа», поэтому используем `find` вместо `find_all`)

In [71]:
summaries = [soup.find('td', class_='FicTbl_sammary').text for soup in fanfic_soups]
summaries[0]

'А, что если Гарри по-настоящему любил только Гермиону, прикрывая дружбой настоящие чувства и Грейнджер видела рядом с собой только Рона, пока тот находился в ее поле зрения. Уизли их оставит и мы посмотрим, как будут развиваться события... Кроме взаимных чувств ребятам еще предстоит открыть большую тайну, о которой далекие предки молчали много веков (два разных мира слились воедино).'

## Персонажи

In [29]:
import re

In [70]:
characters = []
for soup in fanfic_soups:
    single = [char.text for char in soup.find_all(href=re.compile('/character.*'))]
    pairings = [pair.text.split('/') for pair in soup.find_all(href=re.compile('/paring.*'))]
    for pairing in pairings:
        single += pairing
    single = list(set(single))
    characters.append(single)
characters[0]

['Луна Лавгуд',
 'Сириус Блэк',
 'Гермиона Грейнджер',
 'Эдвард Каллен',
 'Рон Уизли',
 'Элис Каллен',
 'Гарри Поттер',
 'Изабелла Свон (Белла)',
 'Северус Снейп',
 'Джейкоб Блэк']

## Пейринги

In [69]:
pairings = []
for soup in fanfic_soups:
    fic_pairings = [pair.text for pair in soup.find_all(href=re.compile('/paring.*'))]
    pairings.append(fic_pairings)
pairings[0]

['Гарри Поттер/Гермиона Грейнджер', 'Изабелла Свон (Белла)/Эдвард Каллен']

## Фандомы

In [68]:
fandoms = []
for soup in fanfic_soups:
    fic_fandoms = [fandom.text for fandom in soup.find_all(href=re.compile('/fandom.*'))]
    fandoms.append(fic_fandoms)
fandoms[0]

['Гарри Поттер', 'Сумерки']

## Возрастной рейтинг

In [80]:
ratings = [soup.find('span', class_='FicTable_Rating').text for soup in fanfic_soups]
ratings[0]

'PG-13'

## Теги отношений

In [112]:
relationships = [soup.find('div', class_='FicTable_Title').find('h4').find('span').text.strip('()') for soup in fanfic_soups]
relationships[0]

'гет'

## Теги размера + размер

In [125]:
sizes_full = [soup.find('span', class_='FicTable_Size').text.split('|') for soup in fanfic_soups]
size_tags = [size_full[0].strip() for size_full in sizes_full]
sizes = [size_full[1].strip() for size_full in sizes_full]
(size_tags[0], sizes[0])

('Макси', '182 Кб')